# Comparing genes

In [ ]:
library(scrabbitr)
library(enrichR)

## Load data

In [ ]:
r_data <- loadRabbitData("../data-in/rabbit/")
m_data <- loadMouseData("../data-in/mouse/")

In [13]:
rm_one2one <- read.table("../data-in/orthologs.tsv",sep="\t")

## Compute marker genes

### Sample cells

In [7]:
# Load precomputed markers
r_markers <- readRDS("../data-out/compare_genes/r_markers.rds")
m_markers <- readRDS("../data-out/compare_genes/m_markers.rds")

## Compare marker genes

In [11]:
ctype_shared = c('Allantois',
 'Allantois endothelium',
 'Anterior cardiopharyngeal progenitors',
 'Anterior somitic tissues',
 'Cardiomyocytes FHF 1',
 'Cardiomyocytes FHF 2',
 'Cardiopharyngeal progenitors',
 'Caudal mesoderm',
 'Cranial mesoderm',
 'Dermomyotome',
 'EMP',
 'EPDC',
 'Embryo proper endothelium',
 'Endocardium',
 'Epiblast',
 'Epicardium',
 'Erythroid',
 'Gut tube',
 'Lateral plate mesoderm',
 'Limb mesoderm',
 'MEP',
 'Megakariocytes',
 'Mesenchyme',
 'Mesothelium',
 'Midgut',
 'Migratory neural crest',
 'NMPs',
 'NMPs/Mesoderm-biased',
 'Nephron progenitors',
 'Neural tube',
 'Node',
 'Non-neural ectoderm 1',
 'Non-neural ectoderm 2',
 'Non-neural ectoderm 3',
 'Non-neural ectoderm 4',
 'Notochord',
 'Otic placode',
 'PGC',
 'Parietal endoderm',
 'Pharyngeal endoderm',
 'Posterior somitic tissues',
 'Presomitic mesoderm',
 'Primitive Streak',
 'Sclerotome',
 'Somitic mesoderm',
 'Thyroid primordium',
 'Venous endothelium',
 'Visceral endoderm',
 'YS endothelium')

In [ ]:
rm_markers <- lapply(ctype_shared,function(celltype){
  rct_markers <- r_markers[[celltype]][,c("p.value","FDR","summary.logFC")]
  mct_markers <- m_markers[[celltype]][,c("p.value","FDR","summary.logFC")]
  
  # Add gene name/ensembl id info
  rct_markers$rgene <- rownames(rct_markers)
  mct_markers$mgene <- rownames(mct_markers)
  
  rct_markers$rensembl <- rabbit.genes[rownames(rct_markers),1]
  mct_markers$mensembl <- mouse.genes[rownames(mct_markers),1]
  
  # Subset one-to-one orthologs
  rct_markers <- rct_markers[rct_markers$rensembl %in% rm_one2one$ref,]
  mct_markers <- mct_markers[mct_markers$mensembl %in% rm_one2one$query,]
  
  rownames(rct_markers) <- sfs[rct_markers$rabbit_ensembl_id,"query"]
  rownames(mct_markers) <- mct_markers$mouse_ensembl_id
  
  # Merge together markers
  markers <- merge(rct_markers,mct_markers,by=0,all=F)
  
  # Remove non-significant degs
  markers <- markers[markers$p.value.x < 0.05 | markers$p.value.y < 0.05,]
  
  markers <- markers[order(markers$p.value.x,-markers$summary.logFC.x,decreasing = F),]
  markers <- markers[,c("rabbit_gene_name","rabbit_ensembl_id","mouse_gene_name",
                        "mouse_ensembl_id","p.value.x","p.value.y","FDR.x","FDR.y",
                        "summary.logFC.x","summary.logFC.y")]
  
  # Assign unique or shared
  markers$class <- ""
  markers$class[(markers$p.value.x < 0.05) & (markers$p.value.y < 0.05) &
                  sign(markers$summary.logFC.x)==sign(markers$summary.logFC.y)] <- "shared"
  markers$class[(markers$p.value.x < 0.05) & (markers$p.value.y > 0.05)] <- "rabbit unique"
  markers$class[(markers$p.value.y < 0.05) & (markers$p.value.x > 0.05)] <- "mouse unique"
  markers$class <- as.factor(markers$class)
  
  return(markers)
  
}) 
names(rm_markers) <- ctype_shared


### Gene set enrichment

In [10]:
dbs <- c("GO_Molecular_Function_2021", "GO_Cellular_Component_2021", "GO_Biological_Process_2021")

if (websiteLive) {
    enriched <- enrichr(c("Runx1", "Gfi1", "Gfi1b", "Spi1", "Gata1", "Kdr"), dbs)
}

## Integration 